In [ ]:
!pip install -q backtesting yfinance
import yfinance as yf
import pandas as pd
from google.colab import drive
# Carpeta donde estan los archivos
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/datosCrypto

# Demas librerias
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import datetime
import random
from datetime import datetime,timedelta
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/datosCrypto


###Machine learning + cryptos (Para entrada)

In [ ]:
## Leemos los datasets que hemos creado!
#path = os.getcwd()+"/NuevoDataset/"
path = os.getcwd()+"/NuevoDataset_Factor3/"
pares = ["BNB","DOGE","ETH","ADA","MATIC","SOL"]
dfProfits = []
for par in pares:
  dfProfits.append(pd.read_csv(path+par+".csv"))
# Vemos la forma
display(dfProfits[0])

## Verificamos si hay valores Nan
for i in range(len(dfProfits)):
  if dfProfits[i].isna().sum().sum()!=0:
    print("Valores NaN en",pares[i])
  else:
    print("No hay valores Nan en",pares[i])

,OpenDate,CloseDate,OpenRate,CloseRate,ProfitRatio
0,20180101,20180116,0.000623,0.000949,52.03
1,20180102,20180116,0.000632,0.000949,49.90
2,20180103,20180116,0.000593,0.000949,59.72
3,20180104,20180116,0.000626,0.000802,27.92
4,20180105,20180202,0.000608,0.000920,51.01
...,...,...,...,...,...
2244,20240223,20240228,0.007460,0.006918,-7.45
2245,20240224,20240228,0.007401,0.006918,-6.71
2246,20240225,20240228,0.007401,0.006918,-6.71
2247,20240226,20240228,0.007513,0.006918,-8.10


No hay valores Nan en BNB
No hay valores Nan en DOGE
No hay valores Nan en ETH
No hay valores Nan en ADA
No hay valores Nan en MATIC
No hay valores Nan en SOL


In [ ]:
# Scripts Para revisar que no haya Huecos!
def generarFechas(fechaIni,fechaFin):
  fechas = []
  fechaIni = datetime.strptime(str(fechaIni), "%Y%m%d")
  fechaFin = datetime.strptime(str(fechaFin), "%Y%m%d")
  for i in range((fechaFin-fechaIni).days+1):
    fechas.append(int((fechaIni+timedelta(days=i)).strftime('%Y%m%d')))
  return fechas

def sinHuecos(dfProfits,crypto):
  bandera = True
  fechaIni = str(int(dfProfits["OpenDate"].iloc[0]))
  fechaFin = str(int(dfProfits["OpenDate"].iloc[-1]))
  fechas = generarFechas(fechaIni,fechaFin)
  #print(crypto,end=",")
  for i in range(len(fechas)):
    if not(fechas[i] in list(dfProfits["OpenDate"])):
      bandera = False
      print('"'+str(fechas[i])+'",')
  return bandera

# Revisamos que no haya huecos
for i in range(len(dfProfits)):
  if sinHuecos(dfProfits[i],pares[i]) == False:
    print("Hay huecos en", pares[i])
  else:
    print("No hay huecos en", pares[i])

No hay huecos en BNB
No hay huecos en DOGE
No hay huecos en ETH
No hay huecos en ADA
No hay huecos en MATIC
No hay huecos en SOL


In [ ]:
# Para convertir una moneda que esta en cambio usd, a una moneda con el cambio en btc
def usdtobtc(moneda,ini,fin):
  df1 = dataCrypto("BTC-USD",ini,fin)
  df2 = dataCrypto(moneda,ini,fin)
  dfdiv = pd.DataFrame()
  dfdiv["Date"] = df1["Date"]
  dfdiv['Open'] = df2['Open'].div(df1['Open'])
  dfdiv['High'] = df2['High'].div(df1['High'])
  dfdiv['Low'] = df2['Low'].div(df1['Low'])
  dfdiv['Close'] = df2['Close'].div(df1['Close'])
  dfdiv['Adj Close'] = df2['Adj Close'].div(df1['Adj Close'])
  dfdiv['Volume'] = df2['Volume']
  return dfdiv

Descargar data crypto y generar el dataset de a 14 dias


In [ ]:
# Para descargar la data de las cryptos
def dataCrypto(crypto, ini,fin):
  start = datetime.strptime(str(ini), "%Y%m%d").strftime("%Y-%m-%d")
  end = datetime.strptime(str(fin), "%Y%m%d").strftime("%Y-%m-%d")
  stock_data = yf.download(crypto, start=start, end=end, progress=False)
  stock_data = stock_data.reset_index()
  return stock_data

# Para generar el dataset mirando de a 14 dias
def dataXY(stock_data,dfProfits,crypto):
  X_windows_all = []
  y_labels_all = []
  dfTrain = stock_data.copy()
  dfTrain = dfTrain[["Open","High","Low","Close","Volume"]]
  print(len(dfTrain),end=",")
  # Cargamos los profits con su fecha
  dfProfits = dfProfits[1:] ########-------------------------------------------------------------------------------------------->
  # Guardamos la salida
  #y =[round(yy/100,4) for yy in list(dfProfits["ProfitRatio"])] # Solucion rapida para dividirlos por 100 #->>>>>>>>>>>>> cambiado
  y =[yy/100 for yy in list(dfProfits["ProfitRatio"])] # Solucion rapida para dividirlos por 100
  print(len(y),end=",")
  # Vamos a poner el nuevo dataframe
  dfClean = dfTrain.diff()
  dfClean = dfClean.div(dfTrain.shift(1), axis='columns', fill_value = 0)##.dropna() # Como saca el de arriba esta bienponer abajo uno menos
  openDate = pd.to_datetime(dfProfits["OpenDate"].astype(str),format = "%Y%m%d")
  closeDate =  pd.to_datetime(dfProfits["CloseDate"].astype(str),format = "%Y%m%d")
  X = dfClean.values
  #----------------------------------UNION----------------------------------#
  # Creamos el set de datos bien con las ventanas movibles
  window_size = 14 # Dias
  X_windows, y_labels, y_labels2 = [], [], [] # y_labels2 regresion, y_labels clasificacion binaria
  for i in range(len(X) - window_size):
      X_window = X[i:i+window_size, :]#.round(4) #->>>>>>>>>>>>> cambiado
      #print(sum(np.isnan(X_window)))
      y_label = y[i+window_size]
      winopenclose = list(np.concatenate([np.concatenate(X_window), np.concatenate([np.array([openDate[i+window_size+1]]), np.array([closeDate[i+window_size+1]])])])) #np.concatenate([np.array([openDate[i+window_size]]), np.array([closeDate[i+window_size]])])]))
      #winopenclose.append(crypto.replace("-USD","")) #CAMBIAMOS A BTC
      winopenclose.append(crypto.replace("-BTC",""))
      X_windows.append(winopenclose)

      y_labels.append(y_label)

  X_windows_all.append(np.array(X_windows)) # Guardamos pero antes los convertimos a array
  y_labels_all.append(np.array(y_labels))

  return X_windows_all,y_labels_all


In [ ]:
## Transformamos la data en un formato entrenable para los modelos a utilizar
# Ponemos las cryptos al cambio que queremos
intercambio = "-BTC"
cryptos = [p+intercambio for p in pares]
cryptos_usd = [p+"-USD" for p in pares]
dfs = dfProfits

#Hacemos una lista de todos los train-val-test
X_all = []
y_all = []
for i in range(len(cryptos)):
  crypto = cryptos[i]
  print(crypto)
  # Fechas
  ini = str(dfs[i]["OpenDate"][0])
  iniVal = "20210701"
  iniTest = "20220301"
  fin = str(list( dfs[i]["OpenDate"])[-1])
  # Pasamos de usd a btc
  stock_dataTrain = usdtobtc(cryptos_usd[i],ini,fin)

  ## Separamos en train, val y test
  X = dataXY(stock_dataTrain,dfs[i],crypto)[0][0]
  y = dataXY(stock_dataTrain,dfs[i],crypto)[1][0]
  # Guardamos
  X_all.append(X)
  y_all.append(y)
  print("")
X_all = np.concatenate(X_all)
y_all = np.concatenate(y_all)

BNB-BTC
2248,2248,2248,2248,
DOGE-BTC
1694,1694,1694,1694,
ETH-BTC
2249,2249,2249,2249,
ADA-BTC
2247,2247,2247,2247,
MATIC-BTC
1764,1764,1764,1764,
SOL-BTC
1414,1414,1414,1414,


In [ ]:
# vemos la cantidad de NaN, si es igual a 30 esta bien ya que es el dia anterior que no hay datos multiplicado por 6 cryptos (volumne high close open low del dia anterior de cada una)
is_nan = lambda x: isinstance(x, float) and np.isnan(x)
cantidad_nan = np.sum(np.vectorize(is_nan)(X_all))
cantidad_nan

30

In [ ]:
## Separamos en TRAIN VAL y TEST
# data x
X_train = [ xx for xx in X_all if xx[70]<pd.to_datetime(iniVal)]
X_val =  [ xx for xx in X_all if (xx[70]>=pd.to_datetime(iniVal) and (xx[70]<pd.to_datetime(iniTest)))]
X_test =  [ xx for xx in X_all if xx[70]>=pd.to_datetime(iniTest)]
# data y
y_train = y_all[0:len(X_train)]
y_val =  y_all[len(X_train):len(X_train)+len(X_val)]
y_test =  y_all[len(X_train)+len(X_val):]

In [ ]:
print("Igualdad de largos para train validacion y test:")
"Train: "+str(len(X_train) == len(y_train)), "Validacion: "+str(len(X_val) == len(y_val)),"Test: "+str(len(X_test) == len(y_test))

Igualdad de largos para train validacion y test:


('Train: True', 'Validacion: True', 'Test: True')

14dias Open high low close volume + columna profit, columna dataseparator y el closeDate y openDate

In [ ]:
# CAMBIAMOS LOS NOMBRES A TRAIN VALIDATE Y TEST!
dataframeFINAL = pd.concat([pd.DataFrame(X_all),pd.DataFrame({"profits":y_all})],axis=1)
dataframeFINAL.columns = dataframeFINAL.columns.astype("str")
dataframeFINAL = dataframeFINAL.rename(columns={"70":"startDate","71":"endDate","72":"Crypto"})
dataframeFINAL["Partitor"]= ""
dataframeFINAL.loc[dataframeFINAL["startDate"] < "20210701", "Partitor"] = "TRAIN"
dataframeFINAL.loc[(dataframeFINAL["startDate"] >= "20210701") & (dataframeFINAL["startDate"] < "20220301"), "Partitor"] = "VALIDATE"
dataframeFINAL.loc[dataframeFINAL["startDate"] >= "20220301", "Partitor"] = "TEST"
dataframeFINAL.to_csv("reinfocercement2/CryptoFinal.csv",index = False)
dataframeFINAL.iloc[:,[0,1,2,3,4,5,-1,-2,-3]]

,0,1,2,3,4,5,Partitor,profits,Crypto
0,NaN,NaN,NaN,NaN,NaN,0.015134,TRAIN,-0.3623,BNB
1,0.015134,-0.041949,0.031712,-0.042591,0.577606,-0.056741,TRAIN,-0.2412,BNB
2,-0.056741,0.04158,-0.079564,0.063454,0.038773,0.056415,TRAIN,-0.2876,BNB
3,0.056415,-0.014971,0.061138,-0.058421,0.459036,-0.044695,TRAIN,-0.2821,BNB
4,-0.044695,0.55809,-0.018816,0.448962,1.150278,0.440983,TRAIN,-0.2950,BNB
...,...,...,...,...,...,...,...,...,...
11527,-0.015098,-0.008248,-0.007646,-0.001011,0.253983,-0.001053,TEST,-0.1198,SOL
11528,-0.001053,-0.004207,-0.016876,0.003712,0.039546,0.003556,TEST,-0.0851,SOL
11529,0.003556,0.011941,0.00957,0.013499,0.019771,0.013461,TEST,-0.0641,SOL
11530,0.013461,0.007154,0.024322,-0.004207,0.320039,-0.003918,TEST,-0.0441,SOL


In [ ]:
# CONTROL COLUMNAS
nan_count_per_column = dataframeFINAL.isna().sum()
print("Cantidad total de valores Nan:", nan_count_per_column.sum()) # 6*5 es igual a 30, es decir el nan del dia anterior de cada uno (hig low open close volume)

inf_count_per_column = dataframeFINAL.isin([np.inf, -np.inf]).sum()
print("Cantidad total de valores infinitos:", inf_count_per_column.sum())

inf_count_per_column = dataframeFINAL.isin([0]).sum()
print("Cantidad total de valores cero:", inf_count_per_column.sum())

Cantidad total de valores Nan: 30
Cantidad total de valores infinitos: 0
Cantidad total de valores cero: 2


In [ ]:
# Concatenamos otras metricas!
## Modificando fechas este es ulitmo
# Train
DataCleanFinal_train = pd.concat([pd.concat([pd.DataFrame(X_train).round(6),pd.DataFrame({"profits":y_train})],axis=1),pd.DataFrame({"Partitor":["TRAIN" for x in range(len(X_train))]})],axis=1)
DataCleanFinal_train.columns = DataCleanFinal_train.columns.astype("str")
# Val
DataCleanFinal_val = pd.concat([pd.concat([pd.DataFrame(X_val).round(6),pd.DataFrame({"profits":y_val})],axis=1),pd.DataFrame({"Partitor":["VALIDATE" for x in range(len(X_val))]})],axis=1)
DataCleanFinal_val.columns = DataCleanFinal_val.columns.astype("str")
# Test
DataCleanFinal_Tes = pd.concat([pd.concat([pd.DataFrame(X_test).round(6),pd.DataFrame({"profits":y_test})],axis=1),pd.DataFrame({"Partitor":["TEST" for x in range(len(X_test))]})],axis=1)
DataCleanFinal_Tes.columns = DataCleanFinal_Tes.columns.astype("str")
## All dataset
AllDs = pd.concat([DataCleanFinal_train,pd.concat([DataCleanFinal_val,DataCleanFinal_Tes],axis=0)],axis=0)
AllDs = AllDs.rename(columns={"70":"startDate","71":"endDate","72":"Crypto"})
AllDs = AllDs.dropna()
AllDs.to_csv("NuevoDataset_Factor3/Data_IntercambioBTC_F3.csv",index = False)

In [ ]:

print("Forma del dataframe:",AllDs.shape)
print("Valores NaN del dataframe final:",AllDs.isna().sum().sum())
print("Porcentaje de profits mayores a cero:",str(round(100*len(AllDs[AllDs["profits"]>0]["profits"])/len(AllDs),2))+"%")

Forma del dataframe: (11526, 75)
Valores NaN del dataframe final: 0
Porcentaje de profits mayores a cero: 37.84%


In [ ]:
AllDs.iloc[:,[4,70]]

,4,startDate
1,0.577606,2018-01-17
2,0.038773,2018-01-18
3,0.459036,2018-01-19
4,1.150278,2018-01-20
5,0.865334,2018-01-21
...,...,...
4360,0.253983,2024-02-20
4361,0.039546,2024-02-21
4362,0.019771,2024-02-22
4363,0.320039,2024-02-23
